In [ ]:
!pip3 install pytorch

In [ ]:
# 필수 라이브러리
import tweepy
from newspaper import Article
from textblob import TextBlob
from transformers import pipeline
import pandas as pd
import tweepy.errors

# 환경 설정
TWITTER_KEYS = {
    'consumer_key': 'PbFmBy4hFyCGL3IAKRRTOiLYi',
    'consumer_secret': 'R2PfC84RomhVkEV7JYaIVWa4XN9b0NptXo2Ke5EEgquiQLrKIF',
    'access_token': '1922186741060141056-Ti4PfL62CJaIHhcbbughOQUYnlJj1e',
    'access_token_secret': '4xZ1XPMNfd1JughpfwdUq9u2yH8Ey7d98vpPrtVrr5gcP'
}

BBC_FOOTBALL_URL = "https://www.bbc.com/sport/football"

def twitter_crawler(query, count=100):
    auth = tweepy.OAuthHandler(TWITTER_KEYS['consumer_key'], 
                              TWITTER_KEYS['consumer_secret'])
    auth.set_access_token(TWITTER_KEYS['access_token'],
                         TWITTER_KEYS['access_token_secret'])
    
    api = tweepy.API(auth)
    tweets = []
    
    try:
        for tweet in tweepy.Cursor(api.search_tweets,
                                  q=query,
                                  tweet_mode='extended',
                                  lang='en').items(count):
            tweet_data = {
                'source': 'Twitter',
                'date': tweet.created_at,
                'content': tweet.full_text,
                'likes': tweet.favorite_count,
                'retweets': tweet.retweet_count
            }
            tweets.append(tweet_data)
    except AttributeError:
        print(f"Twitter API Error: {e}")
    
    return tweets


def bbc_news_crawler(max_articles=10):
    summarizer = pipeline("summarization", model="t5-small")
    articles = []
    
    try:
        # BBC 축구 섹션에서 최신 기사 수집
        football_news = Article(BBC_FOOTBALL_URL)
        football_news.download()
        football_news.parse()
        
        # 기사 링크 추출
        links = [a['href'] for a in football_news.html.find_all('a', href=True)
                if '/sport/football/' in a['href']][:max_articles]
        
        # 개별 기사 처리
        for link in links:
            article = Article(link)
            article.download()
            article.parse()
            
            # 요약 생성
            summary = summarizer(article.text[:1024], 
                                max_length=150, 
                                min_length=30, 
                                do_sample=False)[0]['summary_text']
            
            articles.append({
                'source': 'BBC News',
                'date': article.publish_date,
                'title': article.title,
                'content': article.text,
                'summary': summary
            })
            
    except Exception as e:
        print(f"News Crawling Error: {e}")
    
    return articles

def analyze_sentiment(text):
    analysis = TextBlob(text)
    return {
        'polarity': analysis.sentiment.polarity,
        'subjectivity': analysis.sentiment.subjectivity
    }

def process_data(tweets, articles):
    processed = []
    
    # 트위터 데이터 처리
    for tweet in tweets:
        sentiment = analyze_sentiment(tweet['content'])
        processed.append({
            **tweet,
            **sentiment,
            'type': 'social_media'
        })
    
    # 뉴스 데이터 처리
    for article in articles:
        sentiment = analyze_sentiment(article['content'])
        processed.append({
            **article,
            **sentiment,
            'type': 'news_article'
        })
    
    return pd.DataFrame(processed)

if __name__ == "__main__":
    # 데이터 수집
    tweets = twitter_crawler("PremierLeague OR ChampionsLeague", 50)
    articles = bbc_news_crawler(5)
    
    # 데이터 처리
    df = process_data(tweets, articles)
    
    # 결과 출력
    print(f"Collected {len(df)} items:")
    print(df[['source', 'date', 'polarity', 'summary']].head())
    
    # CSV 저장
    df.to_csv('football_media_analysis.csv', index=False)


AttributeError: module 'tweepy.errors' has no attribute 'TweepError'